# Librerías

In [1]:
import pickle 
import networkx as nx
import seaborn as sns
from networkx.algorithms import community
import numpy as np
from sklearn.decomposition import PCA

# Creación de los grafos
- Carga de datos
- Filtro de correlaciones menores a 10e-5
- Análisis de dispersión de la matriz resultante
- Creación del grafo

In [2]:
with open('../data/correlaciones_macosko/correlaciones.pickle', 'rb') as f:
    correlaciones = pickle.load(f)

In [3]:
# Filtro de correlaciones menores o iguales que 10 e-5
correlaciones = np.where(correlaciones <= 10**(-5), 0, correlaciones)

for i in range(correlaciones.shape[0]):
    correlaciones[i,i] = 0

np.min(correlaciones[correlaciones!=0])

1.0000448157766349e-05

In [4]:
# Porcentaje de valores distintos de 0
len(correlaciones.nonzero()[0])/(correlaciones.shape[0]**2)*100 # NB 47

21.042566688045387

In [34]:
sns.heatmap(correlaciones)

In [5]:
grafo = nx.Graph()

# Agregar nodos al grafo
grafo.add_nodes_from(range(len(correlaciones)))

# Agregar aristas al grafo con pesos
for i in range(len(correlaciones)):
    for j in range(i + 1, len(correlaciones)):
        peso = correlaciones[i, j]
        if peso != 0:
            grafo.add_edge(i, j, weight=peso)

# Detección de comunidades en el grafo

In [6]:
# Ejecutar el algoritmo de Louvain
particiones = nx.community.louvain_communities(grafo, seed=123)

In [7]:
with open('resultados/particiones_grafos_macosko.pickle', 'wb') as f:
    pickle.dump(particiones, f)

# Mirar métricas

### Funciones auxiliares

In [2]:
from sklearn.metrics import (calinski_harabasz_score, davies_bouldin_score, silhouette_score)
from scipy.optimize import linear_sum_assignment
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt 


def cluster_acc_plot(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    y_pred = y_pred.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_sum_assignment(w.max() - w)
    
    df_cm = pd.DataFrame(w, index = [i for i in range(D)], columns = [i for i in range(D)])
    plt.figure(figsize = (10,7))
    w_order = np.zeros((D, D), dtype=np.int64)
    for i in range(D):
        for j in range(D):
            w_order[i,j] = w[i, ind[1][j]]

    df_cm = pd.DataFrame(w_order, index = [i for i in range(D)], columns = [i for i in ind[1]])
    plt.figure(figsize = (10,7))
    sns.heatmap(df_cm, annot=True, fmt='g')
    plt.ylabel("Prediction")
    plt.xlabel("Ground Truth")
    
def cluster_acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    y_pred = y_pred.astype(np.int64)

    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
        
    ind = linear_sum_assignment(w.max() - w)
    ind = np.asarray(ind)
    ind = np.transpose(ind)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

def unsupervised_metrics(X, y_pred):
    # Evaluación final de resultados: métricas comparando con los clusters reales
    sil = np.round(silhouette_score(X, y_pred), 5)
    chs = np.round(calinski_harabasz_score(X, y_pred), 5)
    dbs = np.round(davies_bouldin_score(X, y_pred), 5)
    print('Evaluating cells: SIL= %.4f, CHS= %.4f, DBS= %.4f' % (sil, chs, dbs))

### Resultados poisson

In [23]:
with open('generados/zi__poisson_dim10000/resultados_louvain.pickle', 'rb') as f:
    particiones_p = pickle.load(f)

# ZI Poisson
with open('generados/zi_poisson_dim10000.pkl', 'rb') as f:
    poisson = pickle.load(f)
    X_p = np.array(poisson['X'])
    y_p = np.array(poisson['y'])

diccionario = {}

# Crear el diccionario
for i, conjunto in enumerate(particiones_p):
    for elemento in conjunto:
        diccionario[elemento] = i

# Crear la lista deseada
max_elemento = max(max(particiones_p, key=max), default=-1)
clusters_p = np.array([diccionario.get(i, -1) for i in range(max_elemento + 1)])
print(clusters_p)

[0 0 0 ... 7 7 7]


In [24]:
acc = round(cluster_acc(clusters_p, y_p),3)
nmi = round(metrics.normalized_mutual_info_score(clusters_p, y_p),3)
ari = round(metrics.adjusted_rand_score(clusters_p, y_p),3)

print(f"ACC: {acc}. NMI: {nmi}. ARI: {ari}.")

unsupervised_metrics(X_p, y_p)

ACC: 1.0. NMI: 1.0. ARI: 1.0.
Evaluating cells: SIL= 0.0641, CHS= 197.9162, DBS= 3.6413


In [28]:
X_p_pca = PCA(n_components=32).fit_transform(X_p)
unsupervised_metrics(X_p_pca, y_p)

Evaluating cells: SIL= 0.7464, CHS= 18953.4315, DBS= 0.3673


### Resultados NB

In [32]:
with open('generados/zi_negative_binomial_dim10000/resultados_louvain.pickle', 'rb') as f:
    particiones_nb = pickle.load(f)

# ZI Negative Binomial
with open('generados/zi_negative_binomial_dim10000.pkl', 'rb') as f:
    nb = pickle.load(f)
    X_nb = nb['X']
    y_nb = np.array(nb['y'])

In [33]:
diccionario = {}

# Crear el diccionario
for i, conjunto in enumerate(particiones_nb):
    for elemento in conjunto:
        diccionario[elemento] = i

# Crear la lista deseada
max_elemento = max(max(particiones_nb, key=max), default=-1)
clusters_nb = np.array([diccionario.get(i, -1) for i in range(max_elemento + 1)])
print(clusters_nb)

[2 6 3 ... 2 6 4]


In [35]:
len(clusters_nb)

7875

In [36]:
len(y_nb)

7897

In [34]:
acc = round(cluster_acc(clusters_nb, y_nb),3)
nmi = round(metrics.normalized_mutual_info_score(clusters_nb, y_nb),3)
ari = round(metrics.adjusted_rand_score(clusters_nb, y_nb),3)

print(f"ACC: {acc}. NMI: {nmi}. ARI: {ari}.")

unsupervised_metrics(X_nb, y_nb)

AssertionError: 

### Resultados Human Liver

In [27]:
import h5py 

with open('resultados/particiones_grafos_human_liver.pickle', 'rb') as f:
    particiones_hl = pickle.load(f)

# ZI Negative Binomial
with h5py.File('../data/HumanLiver_counts_top5000.h5') as f:
    X_hl = np.array(f['X'])
    y_hl = np.array(f['Y'])

In [29]:
diccionario = {}

# Crear el diccionario
for i, conjunto in enumerate(particiones_hl):
    for elemento in conjunto:
        diccionario[elemento] = i

# Crear la lista deseada
max_elemento = max(max(particiones_hl, key=max), default=-1)
clusters_hl = np.array([diccionario.get(i, -1) for i in range(max_elemento + 1)])
print(clusters_hl)

[3 3 3 ... 3 0 2]


In [30]:
acc = round(cluster_acc(clusters_hl, y_hl),3)
nmi = round(metrics.normalized_mutual_info_score(clusters_hl, y_hl),3)
ari = round(metrics.adjusted_rand_score(clusters_hl, y_hl),3)

print(f"ACC: {acc}. NMI: {nmi}. ARI: {ari}.")

unsupervised_metrics(X_hl, y_hl)

ACC: 0.766. NMI: 0.793. ARI: 0.779.
Evaluating cells: SIL= -0.2044, CHS= 189.0439, DBS= 2.7284


### Resultados 10X_PBMC

In [15]:
import h5py 

with open('resultados/particiones_grafos_10x.pickle', 'rb') as f:
    particiones_10x = pickle.load(f)

# ZI Negative Binomial
with h5py.File('../data/10X_PBMC_select_2100.h5') as f:
    X_10x = np.array(f['X'])
    y_10x = np.array(f['Y'])

In [16]:
diccionario = {}

# Crear el diccionario
for i, conjunto in enumerate(particiones_10x):
    for elemento in conjunto:
        diccionario[elemento] = i

# Crear la lista deseada
max_elemento = max(max(particiones_10x, key=max), default=-1)
clusters_10x = np.array([diccionario.get(i, -1) for i in range(max_elemento + 1)])
print(clusters_10x)

[1 2 3 ... 0 2 2]


In [17]:
acc = round(cluster_acc(clusters_10x, y_10x),3)
nmi = round(metrics.normalized_mutual_info_score(clusters_10x, y_10x),3)
ari = round(metrics.adjusted_rand_score(clusters_10x, y_10x),3)

print(f"ACC: {acc}. NMI: {nmi}. ARI: {ari}.")

unsupervised_metrics(X_10x, y_10x)

ACC: 0.708. NMI: 0.742. ARI: 0.639.
Evaluating cells: SIL= 0.0223, CHS= 152.8066, DBS= 3.3886


### Resultados Macosko

In [3]:
import h5py 

with open('resultados/particiones_grafos_macosko.pickle', 'rb') as f:
    particiones_mac = pickle.load(f)

# ZI Negative Binomial
with h5py.File('../data/Macosko_mouse_retina.h5') as f:
    X_mac = np.array(f['X'])
    y_mac = np.array(f['Y'])

In [4]:
diccionario = {}

# Crear el diccionario
for i, conjunto in enumerate(particiones_mac):
    for elemento in conjunto:
        diccionario[elemento] = i

# Crear la lista deseada
max_elemento = max(max(particiones_mac, key=max), default=-1)
clusters_mac = np.array([diccionario.get(i, -1) for i in range(max_elemento + 1)])
print(clusters_mac)

[3 1 0 ... 0 1 0]


In [5]:
acc = round(cluster_acc(clusters_mac, y_mac),3)
nmi = round(metrics.normalized_mutual_info_score(clusters_mac, y_mac),3)
ari = round(metrics.adjusted_rand_score(clusters_mac, y_mac),3)

print(f"ACC: {acc}. NMI: {nmi}. ARI: {ari}.")

unsupervised_metrics(X_mac, y_mac)

ACC: 0.441. NMI: 0.529. ARI: 0.394.
Evaluating cells: SIL= -0.0348, CHS= 124.1474, DBS= 5.1253
